In [87]:
import argparse
import json
import os
import pytorch_lightning as L
import numpy as np
import torch
import torchmetrics
import trainUtils
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

In [ ]:
nn.functional.binary_cross_entropy(pos)

In [88]:
path = "/data2/tyfei/trainresults/ionChannels/ESMC/domain885/" 
with open(os.path.join(path, "config.json"), "r") as f:
    configs = json.load(f)

In [89]:
pretrain_model = trainUtils.loadPretrainModel(configs)

model = trainUtils.buildModel(configs, pretrain_model, None)

In [91]:
t = torch.load("/data2/tyfei/trainresults/ionChannels/ESMC/domain885/epoch=7-validate_acc=0.99.ckpt")
model.load_state_dict(t['state_dict'], strict=False)    

_IncompatibleKeys(missing_keys=['esm_model.embed.weight', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.0.weight', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.0.bias', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.1.linear.weight', 'esm_model.transformer.blocks.0.attn.out_proj.linear.weight', 'esm_model.transformer.blocks.0.attn.q_ln.weight', 'esm_model.transformer.blocks.0.attn.k_ln.weight', 'esm_model.transformer.blocks.0.ffn.0.weight', 'esm_model.transformer.blocks.0.ffn.0.bias', 'esm_model.transformer.blocks.0.ffn.1.linear.weight', 'esm_model.transformer.blocks.0.ffn.3.linear.weight', 'esm_model.transformer.blocks.1.attn.layernorm_qkv.0.weight', 'esm_model.transformer.blocks.1.attn.layernorm_qkv.0.bias', 'esm_model.transformer.blocks.1.attn.layernorm_qkv.1.linear.weight', 'esm_model.transformer.blocks.1.attn.out_proj.linear.weight', 'esm_model.transformer.blocks.1.attn.q_ln.weight', 'esm_model.transformer.blocks.1.attn.k_ln.weight', 'esm_model.transformer.blocks.1.

In [92]:
ds = trainUtils.loadDataset(configs)
dl = ds.val_dataloader()

get val loader


In [35]:
for i, j in enumerate(dl):
    print(j)
    break

called new epoch
[{'seq_t': tensor([[ 0, 20, 12, 10,  9,  4,  4,  4, 12,  6,  7,  6,  6, 18, 18,  6,  5, 12,
         18, 10, 19,  4, 12,  8,  6, 12, 12, 14,  7, 15, 18,  6, 12, 14, 11,  6,
         11,  4,  5,  7, 17,  4, 12,  6,  8, 18, 12,  4,  6, 18,  4, 20, 19,  8,
          8,  4,  4, 11,  8, 20, 14,  8,  9, 19, 15,  4, 18, 12,  6, 11,  6, 18,
         23,  6,  5,  4, 11, 11, 18,  8, 11, 18,  8, 19,  9, 11, 18,  5,  4,  7,
         13,  9,  6,  4,  4, 18, 15,  5,  4,  4, 17, 12,  4, 12, 17,  7,  5,  6,
         23,  4, 12, 20,  7, 19, 18,  6, 10, 11,  7,  7,  4,  7, 12, 18, 15,  2]])}, tensor([[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]), {'seq_t': tensor([[ 0, 20, 17, 15, 12, 11, 15,  7, 15, 11, 20,  8, 15, 10,  8, 12,  5,  5,
         12, 12,  7, 18,  8, 20, 20, 19,  8,  6,  7,  8,  4,  5,  5,  9, 10, 17,
         15,  7,  9, 12,  8, 13, 17,  6, 10,  7, 10,  7, 11, 11, 17,  6, 12, 11,
         15,  9,  5,  6, 15, 18, 10, 15,  8,  9, 11, 10, 18,  6,  9, 11, 

In [93]:
dl = DataLoader(ds.val_set, batch_size=1, shuffle=False)

In [94]:
trainer = L.Trainer(devices=[7])
res = trainer.predict(model, dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 3950/3950 [04:19<00:00, 15.22it/s]


In [95]:
acc = torchmetrics.Accuracy(task="binary")
t = 1
num = []
for t in range(21):
    y = [] 
    pre = [] 
    for i in res:
        if t==0:
            pre.append(i[0][0])
            y.append(i[1][0][0])
        elif (i[1][0][t] > -0.5):
            pre.append(i[0][1][t-1])
            y.append(i[1][0][t])
    print(t)
    print(acc(torch.tensor(pre), torch.tensor(y)), torch.sum(torch.stack(y))/len(y))
    num.append((torch.sum(torch.stack(y))/len(y)).item())
num    

0
tensor(0.9904) tensor(0.4937)
1
tensor(0.7721) tensor(0.4491)
2
tensor(0.6507) tensor(0.5962)
3
tensor(0.8374) tensor(0.0823)
4
tensor(0.9588) tensor(0.2078)
5
tensor(0.8786) tensor(0.9002)
6
tensor(0.8169) tensor(0.0947)
7
tensor(0.7937) tensor(0.0484)
8
tensor(0.9918) tensor(0.0185)
9
tensor(0.8153) tensor(0.0710)
10
tensor(0.8488) tensor(0.0211)
11
tensor(0.9758) tensor(0.0010)
12
tensor(0.7801) tensor(0.4045)
13
tensor(0.8716) tensor(0.2199)
14
tensor(0.9122) tensor(0.2914)
15
tensor(0.7906) tensor(0.3482)
16
tensor(0.8054) tensor(0.1405)
17
tensor(0.8916) tensor(0.2956)
18
tensor(0.8722) tensor(0.1831)
19
tensor(0.8085) tensor(0.3882)
20
tensor(0.9174) tensor(0.7464)


[0.49367088079452515,
 0.44907405972480774,
 0.5961934328079224,
 0.08230452984571457,
 0.2078189253807068,
 0.9002057909965515,
 0.09465020895004272,
 0.04835391044616699,
 0.018518518656492233,
 0.07098765671253204,
 0.021090535447001457,
 0.001028806553222239,
 0.40452393889427185,
 0.21988427639007568,
 0.29142555594444275,
 0.3482377827167511,
 0.14045239984989166,
 0.29563388228416443,
 0.18306154012680054,
 0.3882167339324951,
 0.7464492321014404]

In [86]:
0.5/np.array(num)

array([  0.97008542,   1.14705881,   0.87053568,   6.8023253 ,
         2.16666666,   0.55188679,   5.31818171,  10.08620674,
        41.78571352,   7.13414601,  32.49999997, 292.49999751,
         1.25550664,   2.11111106,   1.69642863,   1.36363633,
         3.65384625,   1.69642863,   2.74038468,   1.32558142,
         0.67216981])

tensor(0.5915)

In [10]:
import pickle
with open("/data/tyfei/datasets/ion_channel/Interpro_v2/kingdom/Eukaryota1_label.pkl_v2_1125.pkl", "rb") as f:
    bacteria = pickle.load(f)
bac = bacteria[0]
bac

{'randomseed': 1509,
 'model': 'esm3_sm_open_v1',
 'id': 'A0A151XG23',
 'ori_seq': 'MTISYASEVPNGSSFGCFWRILIKWRGSVYKLIWRELLAYLFFYYLINFTYRYVLNEHQRQIFEKIRYYFGNSSESIPMSFVLGFYVSLVVKRWWEQYKLLPWPDNLALFISAAIPGNDERGRLMRRNIVRYAVLAYVITLQRISLRVKRRFPTLQHMVDVGLMMESEKKIFEMMNKKAAMSKYWMPLVWATNIINRARKEALITSDHVVQTLLVELSDIRKRLGALIGYDTVCVPLVYTQVVTLSLYAYFFSALLGRQFVEHSNGTGKYEEPDMYFPFFTALQFCFYIGWLKVAEVLINPFGEDDDDIELNWLIDRHIKAGYMIVDEMHEEHPELLKDQYWDEVVPKDLPYTVASEHGQHRKSHQDDVYADYESVDTPLVERKKNGWFQRQITRVGSVRSSSTTYSSGGGFFTRNRHNSVYSSPETGGLPQTNNPNLKMSLYDRLVGRKSVRSQRMGRQGTMTKLNSVPVSLKNRPRIPTPDVTKEVIDREQRLALSAANAANIGAGVVGVIPANGHYPTDLSVVVLSPIQETESSPASGKSGAAALAQAVLSPTLTSAGLMTPVTLTPVTMGHLTQLGLMTTATTTTPHLNNQSNSVQATLTEVTSSEEEGSGSGSSRSGSITGQEDRSTPLIDNNNSPMGSNGSSPIFDGYNDRSPILMRPEKLGYVVTAAVPGIQTNRAMAIDARGRRSASLPGPPVSPAQLSNDRSMSLPQSPGLQSREIRIASVSSRQDGLAIDRLITGSHNQGIRSRTNSFGHDLSRSNQRGQDASRKISTVSCTNASLSSSTTSPTSSTSISTTTVTSSKRGEVYV',
 'seq_t': array([ 0, 20, 11, 12,  8, 19,  5,  8,  9,  7, 14, 17,  6,  8,  8, 18,  6,
        23, 18

In [16]:
bac["seq_t"][33:55], bac["seq_t"][68:90]

(array([12, 22, 10,  9,  4,  4,  5, 19,  4, 18, 18, 19, 19,  4, 12, 17, 18,
        11, 19, 10, 19,  7]),
 array([19, 19, 18,  6, 17,  8,  8,  9,  8, 12, 14, 20,  8, 18,  7,  4,  6,
        18, 19,  7,  8,  4]))

In [20]:
len(baseds._augmentsample(bac, (-1, -1), 100)["seq_t"])

102

In [7]:
dl = ds.train_dataloader() 
for i, d in enumerate(dl):
    print(d) 
    if i>10:
        break

get train loader
called new epoch
[{'seq_t': tensor([[ 0,  9,  9, 13, 11,  9,  8, 13,  4,  4,  5,  7, 17, 12, 12, 17, 10,  8,
          6, 12,  4,  6,  8,  4,  4, 14,  4,  9, 18, 21, 14, 11,  4, 10,  9, 11,
         10,  7, 14, 10, 14,  5,  8, 16,  7,  5, 15, 10,  5, 16, 16,  6,  2]])}, tensor([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1]]), {'seq_t': tensor([[ 0, 20, 20, 15, 19, 15,  4,  4,  8, 15, 16, 15,  6, 18, 12,  8,  4,  6,
          5, 12, 15,  8, 18, 18, 16, 18,  8, 17, 12,  8,  7, 19,  4, 12, 12,  6,
         11, 12,  5,  4, 19,  6, 22, 19, 10, 12, 17,  8,  4,  9,  8,  9, 17, 16,
          9,  4, 16,  8, 15, 17,  5, 13,  4, 17,  7,  9,  7,  9,  9, 11, 15, 15,
          9, 19, 17, 13,  4,  7, 13, 13, 12, 17,  9, 18, 10, 15, 16, 12, 17, 16,
         13, 12, 15, 15, 16, 16, 17, 12, 11, 15,  9,  4, 10,  9, 11, 10, 12, 15,
         17, 16, 15,  9, 12,  8, 15,  4, 10, 16, 12, 18, 17, 16,  8,  8,  8,  6,
          9, 16, 10, 13, 18,  

In [6]:
d[0]["seq_t"].shape

torch.Size([1, 120])

In [7]:
model.forward(d[0])

tensor([[ 0, 20,  5, 10, 18,  7,  5,  4,  7,  4,  4,  6,  4,  4,  8,  4,  8,  9,
          4, 13,  5, 12, 16, 10, 10, 14, 15, 12, 16,  7, 19,  8, 10, 21, 14, 14,
          9, 13,  6, 15, 14, 17, 19,  4, 17, 23, 19,  7,  8,  6, 18, 21, 14, 14,
         16, 12,  9, 12, 13,  4,  4, 15, 17,  6,  9, 15, 12, 15,  8,  9, 16,  8,
         13,  4,  8, 18,  8, 15, 13, 22,  8, 18, 19,  4,  4,  8, 21,  5,  9, 18,
         11, 14, 17,  8, 15, 13, 16, 19,  8, 23, 10,  7, 15, 21,  7, 11,  4,  9,
         16, 14, 16, 12,  7, 15, 22, 13, 10, 13,  4,  2]]) torch.Size([1, 120])


RuntimeError: mat1 and mat2 must have the same dtype, but got Float and BFloat16

In [10]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

In [11]:
protein = ESMProtein(sequence="AAAAA")
client = ESMC.from_pretrained("esmc_300m").to("cuda") # or "cpu"
protein_tensor = client.encode(protein)
logits_output = client.logits(
   protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)
print(logits_output.logits, logits_output.embeddings)

ForwardTrackData(sequence=tensor([[[-38.0000, -38.0000, -38.0000,  12.6250,  21.6250,  22.2500,  22.0000,
           21.7500,  21.3750,  21.6250,  21.6250,  21.2500,  20.7500,  21.3750,
           21.8750,  20.8750,  20.7500,  20.2500,  20.3750,  20.1250,  21.7500,
           19.8750,  19.7500,  19.2500,  18.2500,   1.2656,  -1.6719,  -3.9688,
          -20.6250, -38.2500, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000,
          -38.0000, -38.2500, -38.0000, -38.2500, -38.0000, -38.0000, -38.2500,
          -38.0000, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000, -38.2500,
          -38.0000, -38.2500, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000,
          -38.0000, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000, -38.0000,
          -38.0000],
         [-40.2500, -40.0000, -40.2500,   5.3438,  20.1250,  19.8750,  18.7500,
           20.6250,  18.7500,  18.3750,  18.6250,  18.5000,  18.2500,  18.0000,
           18.7500,  17.7500,  17.3750,  17.1250,  17.7500,  16.8750,  22

In [13]:
client(sequence_tokens=d[0]["seq_t"].cuda())

ESMCOutput(sequence_logits=tensor([[[-36.2500, -36.2500, -36.2500,  ..., -36.2500, -36.2500, -36.2500],
         [-36.5000, -36.5000, -36.5000,  ..., -36.5000, -36.5000, -36.5000],
         [-32.7500, -32.7500, -32.7500,  ..., -32.7500, -32.7500, -32.7500],
         ...,
         [-34.7500, -34.5000, -34.5000,  ..., -34.5000, -34.5000, -34.5000],
         [-30.0000, -30.0000, -30.0000,  ..., -30.0000, -30.0000, -29.8750],
         [-31.0000, -31.0000, -31.0000,  ..., -31.0000, -31.0000, -31.0000]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<ViewBackward0>), embeddings=tensor([[[ 0.0026,  0.0060,  0.0051,  ...,  0.0038, -0.0041, -0.0036],
         [-0.0266,  0.0315,  0.0239,  ..., -0.0066, -0.0077, -0.0089],
         [-0.0270,  0.0322, -0.0048,  ..., -0.0430, -0.0479,  0.0398],
         ...,
         [-0.0444,  0.0344,  0.0262,  ...,  0.0153, -0.0640,  0.0262],
         [-0.0535,  0.0649, -0.0220,  ..., -0.0165, -0.0659,  0.0044],
         [-0.0203,  0.0305,  0.0304,  ..., 

In [ ]:
from esm.utils.sampling import _BatchedESMProteinTensor

In [7]:
client.forward(sequence_tokens = protein_tensor.sequence.unsqueeze(0))

ESMCOutput(sequence_logits=tensor([[[-38.0000, -38.0000, -38.0000,  12.6250,  21.6250,  22.2500,  22.0000,
           21.7500,  21.3750,  21.6250,  21.6250,  21.2500,  20.7500,  21.3750,
           21.8750,  20.8750,  20.7500,  20.2500,  20.3750,  20.1250,  21.7500,
           19.8750,  19.7500,  19.2500,  18.2500,   1.2656,  -1.6719,  -3.9688,
          -20.6250, -38.2500, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000,
          -38.0000, -38.2500, -38.0000, -38.2500, -38.0000, -38.0000, -38.2500,
          -38.0000, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000, -38.2500,
          -38.0000, -38.2500, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000,
          -38.0000, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000, -38.0000,
          -38.0000],
         [-40.2500, -40.0000, -40.2500,   5.3438,  20.1250,  19.8750,  18.7500,
           20.6250,  18.7500,  18.3750,  18.6250,  18.5000,  18.2500,  18.0000,
           18.7500,  17.7500,  17.3750,  17.1250,  17.7500,  16.8750,  2